In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

In [34]:
df = pd.read_csv("../data/selma_test.csv")
df.head()

,Unnamed: 0,Name,Year,Countries,Genres,Budget(USD),Domestic(USD),Domestic_Percentage,Foreign(USD),Foreign_Percentage,Worldwide(USD),Runtime(mins),Rating,Audience_Score,Critics_Score
0,0,10 cloverfield lane,2016,united states,"drama, horror, sci-fi, thriller",5000000,72082999,0.654010,38134000,0.345990,110216999,103,pg-13,0.79,0.91
1,1,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000,38178166,0.713446,15334203,0.286554,53512369,97,pg-13,0.69,0.71
2,2,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...",105000000,94784201,0.351333,175000000,0.648667,269784201,109,pg-13,0.37,0.10
3,3,12 monkeys,1995,united states,"mystery, sci-fi, thriller",29000000,57141459,0.338437,111698000,0.661563,168839459,129,r,0.88,0.88
4,4,12 rounds,2009,united states of america,"action thrillers, thriller, action, crime fiction",20000000,12234694,0.708013,5045632,0.291987,17280326,108,pg-13,0.45,0.31


In [35]:
df['Year'].value_counts()

Year
2016    186
2011    185
2015    176
2014    167
2006    164
       ... 
1940      1
1960      1
1952      1
1966      1
1956      1
Name: count, Length: 74, dtype: int64

In [44]:
average_percentages = df.groupby('Year')[['Foreign_Percentage', 'Domestic_Percentage']].mean()
average_percentages = average_percentages.reset_index()
average_percentages = average_percentages[(average_percentages['Year'] >= 2000) & (average_percentages['Year'] <= 2019)]


In [45]:
average_percentages.head()

,Year,Foreign_Percentage,Domestic_Percentage
49,2000,0.368831,0.631169
50,2001,0.389553,0.610447
51,2002,0.369010,0.630990
52,2003,0.419857,0.580143
53,2004,0.408921,0.591079


In [46]:
fig = px.line(average_percentages, 
            x='Year', 
            y=['Foreign_Percentage', 'Domestic_Percentage'], 
            title='Average Percentages Over Time',
            labels={'value': 'Percentage', 'variable': 'Type'}
)


In [47]:
fig.show()